<a href="https://colab.research.google.com/github/RachanaValluri/Virtual-Sign-Language-Translator-Hand--gesture-/blob/main/Project_1(Hand_Gesture).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install livelossplot

In [ ]:
pip install mediapipe

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import PIL
from collections import defaultdict
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import shutil
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense
from keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from livelossplot import PlotLossesKeras
from sklearn.metrics import confusion_matrix, classification_report
import cv2
import mediapipe as mp
from PIL import Image

In [ ]:
base = '/kaggle/input/hand-gesture-recognition-dataset-one-hand/Dataset_RGB/Dataset_RGB'

In [ ]:
st = {}
for i in os.listdir(base):
    st[i] = len(os.listdir(os.path.join(base, i)))

In [ ]:
count = pd.DataFrame(st.values(), index=st.keys())
count.columns=['count']

In [ ]:
sns.barplot(count,x=count.index, y='count')
fig = plt.gcf()
fig.set_size_inches(10,4)
ax = plt.gca()
for i in ax.containers:
    plt.bar_label(i)
plt.title('class distribution\n')
ax.spines[['top','right']].set_visible(False);

### take a look at the images

In [ ]:
for i in os.listdir(base):
    imgs = os.listdir(os.path.join(base, i))
    sample_imgs = random.sample(imgs, 6)

    fig, axes = plt.subplots(1,6, figsize=(25,4), subplot_kw=dict(xticks=[], yticks=[]))
    for num, ax in enumerate(axes.flat):
        ax.imshow(plt.imread(os.path.join(base, i, sample_imgs[num])))
        ax.set_title(i, size=15)

In [ ]:
sample = '/kaggle/input/hand-gesture-recognition-dataset-one-hand/Dataset_RGB/Dataset_RGB/four/img 1.jpg'

In [ ]:
img = cv2.imread(sample)
black = np.zeros(img.shape, dtype="uint8")
imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
mpHands = mp.solutions.hands
hands = mpHands.Hands(static_image_mode=False,
                      max_num_hands=2,
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5)
mpDraw = mp.solutions.drawing_utils

results = hands.process(imgRGB)
if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            for id, lm in enumerate(handLms.landmark):
                # print(id,lm)
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                cv2.circle(black, (cx, cy), 3, (255, 0, 255), cv2.FILLED)
            mpDraw.draw_landmarks(black, handLms, mpHands.HAND_CONNECTIONS)

plt.imshow(black[:,:,::-1])
plt.axis('off');

### split the dataset

In [ ]:
def make_dir(path):
    if not os.path.exists(path):
        os.mkdir(path)

In [ ]:
output_dir = '/kaggle/working'

In [ ]:
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')
test_dir = os.path.join(output_dir, 'test')

In [ ]:
make_dir(train_dir)
make_dir(val_dir)
make_dir(test_dir)

In [ ]:
for i in os.listdir(base):
    for j in ['train', 'val','test']:
        make_dir(os.path.join(j,i))

In [ ]:
for i in os.listdir(base):
    total_imgs = os.listdir(os.path.join(base, i))
    Xtrain, Xvaltest = train_test_split(total_imgs, test_size=0.3)
    Xval, Xtest = train_test_split(Xvaltest, test_size=0.5)

    for tr in Xtrain:
        sample = os.path.join(base, i, tr)
        img = cv2.imread(sample)
        black = np.zeros(img.shape, dtype="uint8")
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(imgRGB)
        if results.multi_hand_landmarks:
                for handLms in results.multi_hand_landmarks:
                    for id, lm in enumerate(handLms.landmark):
                        # print(id,lm)
                        h, w, c = img.shape
                        cx, cy = int(lm.x * w), int(lm.y * h)
                        cv2.circle(black, (cx, cy), 3, (255, 0, 255), cv2.FILLED)
                    mpDraw.draw_landmarks(black, handLms, mpHands.HAND_CONNECTIONS)

        skeleton_img = Image.fromarray(black)
        skeleton_img.save(os.path.join(train_dir, i, tr))




    for v in Xval:
        sample = os.path.join(base, i, v)
        img = cv2.imread(sample)
        black = np.zeros(img.shape, dtype="uint8")
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(imgRGB)
        if results.multi_hand_landmarks:
                for handLms in results.multi_hand_landmarks:
                    for id, lm in enumerate(handLms.landmark):
                        # print(id,lm)
                        h, w, c = img.shape
                        cx, cy = int(lm.x * w), int(lm.y * h)
                        cv2.circle(black, (cx, cy), 3, (255, 0, 255), cv2.FILLED)
                    mpDraw.draw_landmarks(black, handLms, mpHands.HAND_CONNECTIONS)

        skeleton_img = Image.fromarray(black)
        skeleton_img.save(os.path.join(val_dir, i, v))




    for te in Xtest:
        sample = os.path.join(base, i, te)
        img = cv2.imread(sample)
        black = np.zeros(img.shape, dtype="uint8")
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(imgRGB)
        if results.multi_hand_landmarks:
                for handLms in results.multi_hand_landmarks:
                    for id, lm in enumerate(handLms.landmark):
                        # print(id,lm)
                        h, w, c = img.shape
                        cx, cy = int(lm.x * w), int(lm.y * h)
                        cv2.circle(black, (cx, cy), 3, (255, 0, 255), cv2.FILLED)
                    mpDraw.draw_landmarks(black, handLms, mpHands.HAND_CONNECTIONS)

        skeleton_img = Image.fromarray(black)
        skeleton_img.save(os.path.join(test_dir, i, te))

In [ ]:
for i in os.listdir(base):
    print(f'there are {len(os.listdir(os.path.join(train_dir, i)))} images in the train dataset for class [{i}]')
    print(f'there are {len(os.listdir(os.path.join(val_dir, i)))} images in the val dataset for class [{i}]')
    print(f'there are {len(os.listdir(os.path.join(test_dir, i)))} images in the test dataset for class [{i}]\n')

In [ ]:
for i in os.listdir(train_dir):
    imgs = os.listdir(os.path.join(train_dir, i))
    sample_imgs = random.sample(imgs, 6)

    fig, axes = plt.subplots(1,6, figsize=(25,4), subplot_kw=dict(xticks=[], yticks=[]))
    for num, ax in enumerate(axes.flat):
        ax.imshow(plt.imread(os.path.join(train_dir, i, sample_imgs[num]))[:,:,::-1])
        ax.set_title(i, size=15)

### build the model

In [ ]:
train_generator = ImageDataGenerator(rescale=1.0 / 255,
                                     horizontal_flip=True,
                                     zoom_range=0.1,
                                     shear_range=0.2,
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     rotation_range=4,
                                     vertical_flip=False)

validation_generator = ImageDataGenerator(rescale=1./255)

test_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
train_data_gen = train_generator.flow_from_directory(train_dir,
                                                     class_mode='categorical',
                                                     target_size=(48,48),
                                                     batch_size=32,shuffle=True)

validation_data_gen = validation_generator.flow_from_directory(val_dir,
                                                     class_mode='categorical',
                                                     target_size=(48,48),
                                                     batch_size=32,shuffle=False)

test_data_gen = test_generator.flow_from_directory(test_dir,
                                                     class_mode='categorical',
                                                     target_size=(48,48),
                                                     batch_size=32,shuffle=False)

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = 3,  padding='same', activation = 'relu', input_shape = (48,48,3)))
model.add(MaxPooling2D(pool_size = 2))
model.add(Conv2D(filters = 64, kernel_size = 3,  padding='same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Conv2D(filters = 256, kernel_size = 3,  padding='same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Flatten())
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dense(15,activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
epochs = 20
steps_per_epoch = train_data_gen.n//32
validation_steps = validation_data_gen.n//32

checkpoint = ModelCheckpoint('model_weights.h5', monitor='val_accuracy',save_weight_only=True, mode='max', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.00001, mode='auto')
callbacks=[PlotLossesKeras(), checkpoint, reduce_lr]

history = model.fit(x=train_data_gen,
                    steps_per_epoch = steps_per_epoch,
                    epochs=epochs,
                    validation_data=validation_data_gen,
                    validation_steps=validation_steps,
                    callbacks=callbacks)

### test the model

In [ ]:
model.evaluate(test_data_gen)

In [ ]:
pred = model.predict(test_data_gen, test_data_gen.n//test_data_gen.batch_size)

In [ ]:
sns.heatmap(confusion_matrix(np.argmax(pred,axis=1), test_data_gen.classes), annot=True,
            xticklabels=test_data_gen.class_indices.keys(),
           yticklabels=test_data_gen.class_indices.keys(), cbar=False, cmap='viridis')
fig=plt.gcf()
fig.set_size_inches(10,6);

In [ ]:
print(classification_report(np.argmax(pred,axis=1), test_data_gen.classes))

In [ ]:
model_json = model.to_json()
with open('model_json', 'w') as json_file:
    json_file.write(model_json)